In [499]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

In [500]:
imdb_path = r'C:\Users\espym.LAPTOP-41F90NSA\Desktop\zippedData\im.db\im.db'
conn = sqlite3.connect(imdb_path)


In [501]:
file_path1 = r'C:\Users\espym.LAPTOP-41F90NSA\projects\Movie_Project-1\tmdb.movies.csv.gz'
tmdbDF = pd.read_csv(file_path1)

In [502]:
filepath2 = r'C:\Users\espym.LAPTOP-41F90NSA\projects\Movie_Project-1\tn.movie_budgets.csv.gz'
tn_movieDF = pd.read_csv(filepath2)

In [503]:
filepath3 = r'C:\Users\espym.LAPTOP-41F90NSA\projects\Movie_Project-1\rt.movie_info.tsv.gz'
movie_infoDF = pd.read_csv(filepath3, delimiter='\t')

In [504]:
filepath4 =r'C:\Users\espym.LAPTOP-41F90NSA\projects\Movie_Project-1\rt.reviews.tsv.gz'
reviewsDF = pd.read_csv(filepath4, delimiter='\t', encoding='latin-1')

## Preview of Information in the data

Here we just want to have a look at whats contained in the IMDB dataset and just have a proper feel of whats contained in the dataset

In [505]:
#from the imdb database we first want to know the names and number of the tables.
query0 = ("""
    SELECT name
    AS 'Table_Name'
    FROM sqlite_master
    WHERE type = 'table'
;
""")
pd.read_sql(query0,conn)

,Table_Name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [506]:
# checking for the information contained in the movie basics table
query1 = ("""
    SELECT * 
    FROM movie_basics
    ;
          """)
pd.read_sql(query1,conn).head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [507]:
# In the reviews dataframe we will check for the last values
pd.read_sql(query1,conn).tail()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None
146143,tt9916754,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,2013,NaN,Documentary


In [508]:
#check the shape of the dataframe 
pd.read_sql(query1,conn).shape

(146144, 6)

In [509]:
#check for all the columns in the DF
pd.read_sql(query1,conn).columns

Index(['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres'],
      dtype='object')

In [510]:
pd.read_sql(query1,conn).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [511]:
pd.read_sql(query1,conn).describe()

,start_year,runtime_minutes
count,146144.000000,114405.000000
mean,2014.621798,86.187247
std,2.733583,166.360590
min,2010.000000,1.000000
25%,2012.000000,70.000000
50%,2015.000000,87.000000
75%,2017.000000,99.000000
max,2115.000000,51420.000000


In [512]:
# Checking for the unique genres
df = pd.read_sql(query1,conn)
genre_column = df['genres']
df['genres'].unique()

array(['Action,Crime,Drama', 'Biography,Drama', 'Drama', ...,
       'Music,Musical,Reality-TV', 'Animation,Crime',
       'Adventure,History,War'], dtype=object)

Now that we have a feel of the movie_basics, well also explore movie_ratings table

In [513]:
query2 = ("""
    SELECT * 
    FROM movie_ratings
    ;
        """)
pd.read_sql(query2,conn).head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [514]:
pd.read_sql(query2,conn).tail()

,movie_id,averagerating,numvotes
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5
73855,tt9894098,6.3,128


In [515]:
pd.read_sql(query2,conn).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [516]:
pd.read_sql(query2,conn).describe()

,averagerating,numvotes
count,73856.000000,7.385600e+04
mean,6.332729,3.523662e+03
std,1.474978,3.029402e+04
min,1.000000,5.000000e+00
25%,5.500000,1.400000e+01
50%,6.500000,4.900000e+01
75%,7.400000,2.820000e+02
max,10.000000,1.841066e+06


In [517]:
# previwing the third dataset Movie budgets
tn_movieDF.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [518]:
tn_movieDF.tail()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0
5781,82,"Aug 5, 2005",My Date With Drew,"$1,100","$181,041","$181,041"


In [519]:
tn_movieDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [520]:
tn_movieDF.count()

id                   5782
release_date         5782
movie                5782
production_budget    5782
domestic_gross       5782
worldwide_gross      5782
dtype: int64

## Data Cleaning

Now that we have a feeling of the data, we will now combine the two tables i.e movie_basics and movie_ratings using their primary key movie_id

In [521]:
#Joining the tables using the movie_id column
query3 = ("""
SELECT movie_id, primary_title,start_year,runtime_minutes,genres,averagerating AS 'movie_rating'
FROM movie_basics
JOIN movie_ratings
USING (movie_id)
;
""")
df = pd.read_sql(query3,conn)

In [522]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   start_year       73856 non-null  int64  
 3   runtime_minutes  66236 non-null  float64
 4   genres           73052 non-null  object 
 5   movie_rating     73856 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 3.4+ MB


We will now go ahead and clean the dataset to remove the missing values in the dataframe. The columns runtime_minutes and genres appear to having missing values

In [523]:
df.isnull().sum()

movie_id              0
primary_title         0
start_year            0
runtime_minutes    7620
genres              804
movie_rating          0
dtype: int64

In [524]:
#checking for the percentage of missing values in the dataframe
#checking for the sum of null values
null_count = df.isnull().sum()


In [525]:
#checking for all the values
non_null_count = df.shape[0]

In [526]:
#checking the percentage of the null values to determine if to drop them
null_percentage = (null_count / non_null_count) * 100
print(null_percentage)

movie_id            0.000000
primary_title       0.000000
start_year          0.000000
runtime_minutes    10.317374
genres              1.088605
movie_rating        0.000000
dtype: float64


In [527]:
#The null values take a minimal percentage of the total values and can be droppped
df.dropna(inplace=True)
df.shape

(65720, 6)

In [528]:
#checking null values in the other dataframe
budgetDF = tn_movieDF
budgetDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


The budget dataframe contains no null values

Conversion of datatypes

In [529]:
budgetDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [530]:
#The prodution_budget,domestic_gross and worldwide_gross are denoted as objects and we need them as float in order to mathematical computations
budgetDF['production_budget'] = budgetDF['production_budget'].str.replace('$', '').str.replace(',', '').astype(float)
budgetDF['domestic_gross'] = budgetDF['domestic_gross'].str.replace('$', '').str.replace(',', '').astype(float)
budgetDF['worldwide_gross'] = budgetDF['worldwide_gross'].str.replace('$', '').str.replace(',', '').astype(float)

Checking for duplicates 


In [531]:
df.duplicated().sum()

0

In [532]:
budgetDF.duplicated().sum()

0

From both the datasets, there are no duplicated values

Checking for data consistency

In [533]:
df.describe()

,start_year,runtime_minutes,movie_rating
count,65720.000000,65720.000000,65720.000000
mean,2014.258065,94.732273,6.320902
std,2.600143,209.377017,1.458878
min,2010.000000,3.000000,1.000000
25%,2012.000000,81.000000,5.500000
50%,2014.000000,91.000000,6.500000
75%,2016.000000,104.000000,7.300000
max,2019.000000,51420.000000,10.000000


From this we have a few issues in regards to runtime
1. The minimum runtime is 3 minutes which is quite low to meet a movie threshold
2. The maximum runtime is 51420 which an extreme value

filtering the values to include a minimum threshold of 20 minutes and 180 minutes 

In [534]:
clean_runtime = df[(df['runtime_minutes'] >= 20) & (df['runtime_minutes'] <= 180)]
sum_of_movies_within_threshold = len(clean_runtime)
print('The movies within the threshold are:', sum_of_movies_within_threshold)

The movies within the threshold are: 65365


In [535]:
df = df[(df['runtime_minutes'] >= 20) & (df['runtime_minutes'] <= 180)]

In [536]:
df.describe()

,start_year,runtime_minutes,movie_rating
count,65365.00000,65365.000000,65365.000000
mean,2014.25610,93.270986,6.315686
std,2.59996,21.512739,1.457271
min,2010.00000,20.000000,1.000000
25%,2012.00000,81.000000,5.500000
50%,2014.00000,91.000000,6.500000
75%,2016.00000,104.000000,7.300000
max,2019.00000,180.000000,10.000000


In [537]:
budgetDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5782 non-null   int64  
 1   release_date       5782 non-null   object 
 2   movie              5782 non-null   object 
 3   production_budget  5782 non-null   float64
 4   domestic_gross     5782 non-null   float64
 5   worldwide_gross    5782 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 271.2+ KB


We have an issue with the release date as its an object and its needed to be in date format

In [538]:
#conversion of release date from object to datetime
budgetDF['release_date'] = pd.to_datetime(budgetDF['release_date'])

In [539]:
budgetDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   float64       
 4   domestic_gross     5782 non-null   float64       
 5   worldwide_gross    5782 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 271.2+ KB


In [540]:
#checking for the duration of info in the budget dataframe
start_date = budgetDF['release_date'].min()
end_date = budgetDF['release_date'].max()

print(f'the start date is {start_date} and the end date is {end_date}')


the start date is 1915-02-08 00:00:00 and the end date is 2020-12-31 00:00:00


From this we see the first date is 1915 which is significantly a long time ago and the film and production industry has changed in one way or another. We will now convert the dataframe to the year 2000

In [550]:
budgetDF = budgetDF[budgetDF['release_date'] >= '2000-1-1 00:00:00']

## Explarotary Analysis

In [545]:
#checking for the year and duration of the release dates 
start_date = df['start_year'].min()
end_date = df['start_year'].max()
period_taken = end_date - start_date

print('start:', start_date)
print('stop:', end_date)
print('durarion:', period_taken,'years')

start: 2010
stop: 2019
durarion: 9 years


In [ ]:
#checking for the duration of info in the budget dataframe
start_date = budgetDF['release_date'].min()
end_date = budgetDF['release_date'].max()
period_taken = end_date -  start_date

print('The start date is', start_date)
print('The end date id', end_date)
print('Period of the dataset is', period_taken)

The start date is 2000-01-12 00:00:00
The end date id 2020-12-31 00:00:00
Period of the dataset is 7659 days 00:00:00


Univariate Analysis

In [556]:
df.columns

Index(['movie_id', 'primary_title', 'start_year', 'runtime_minutes', 'genres',
       'movie_rating'],
      dtype='object')

In [557]:
df['primary_title'].describe()

count          65365
unique         62118
top       The Return
freq              10
Name: primary_title, dtype: object

In [547]:
#Checking for the most common genre
df['genres'].describe()


count     65365
unique      905
top       Drama
freq      10137
Name: genres, dtype: object

In [548]:
budgetDF.columns

Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')

In [554]:
budgetDF['release_date'].describe()

<ipython-input-554-5a1bd0f6f041>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  budgetDF['release_date'].describe()


count                    4387
unique                   1459
top       2014-12-31 00:00:00
freq                       24
first     2000-01-12 00:00:00
last      2020-12-31 00:00:00
Name: release_date, dtype: object